In [1]:
import pandas as pd

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：2


['https://www.4s.com/en/ecatalog?part=Evaporator%20Core&type=p&search=s',
 'https://www.4s.com/en/ecatalog?part=Heater%20Core&type=p&search=s']

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

url = input_.loc[a, 'Url']

dict_param = {}
list_param = url.split('?')[1].split('&') if '?' in url else []
for param in list_param:
    dict_param[param.split('=')[0]] = param.split('=')[1]

output, page = pd.DataFrame(), 0
while True:
    b = 0
    while True:
        b += 1
        print(b, page)
        try:
            request_url = f'''https://ecatalog.smpcorp.com/v2/fs/api/part/partsearch?filter={dict_param['part']}&filterType={dict_param.get('search', 'n')}&searchType={dict_param.get('type', 'null')}&imageSize=80&start={page*96}&limit=96&sort=3&catFilter=-All-&yearFilter=-All-&makeFilter=-All-&modelFilter=-All-&engineFilter=-All-&attrCodeFilter=-All-&attrValueFilter=-All-&plkEngineMakeFilter=-All-&plkEngineModelFilter=-All-&plkEngineDispFilter=-All-'''
            
            resp = requests.get(request_url,
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(10, 10))

            if resp.status_code == 200:
                list_dict = resp.json()
                if list_dict:
                    break
        except KeyboardInterrupt:
            break
        except:
            continue

    # = = = = = = = = = = = = = = =

    if page == 0:
        count = list_dict[0]['maxRows']

    # = = = = = = = = = = = = = = =

    df_temp = pd.DataFrame({'No': [dict_['rowId'] for dict_ in list_dict],
                            'Part Number': [dict_['basePart'].strip() for dict_ in list_dict],
                            'Url': [f'''https://www.4s.com/en/ecatalog?part={dict_['basePart']}&type=p&search=s''' for dict_ in list_dict]})

    output = pd.concat([output, df_temp], ignore_index=True).fillna('')
    
    # = = = = = = = = = = = = = = =

    page += 1

    # = = = = = = = = = = = = = = =

    if len(output) == count:
        output = output.sort_values(by=['No'],
                                    ascending=[True],
                                    ignore_index=True)
        
        output.to_excel('./test_part_number.xlsx', index=False)
        
        break

output

1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7


,No,Part Number,Url
0,1,16149,https://www.4s.com/en/ecatalog?part=16149&type...
1,2,16152,https://www.4s.com/en/ecatalog?part=16152&type...
2,3,16155,https://www.4s.com/en/ecatalog?part=16155&type...
3,4,44001,https://www.4s.com/en/ecatalog?part=44001&type...
4,5,44002,https://www.4s.com/en/ecatalog?part=44002&type...
...,...,...,...
716,717,64176,https://www.4s.com/en/ecatalog?part=64176&type...
717,718,64177,https://www.4s.com/en/ecatalog?part=64177&type...
718,719,64178,https://www.4s.com/en/ecatalog?part=64178&type...
719,720,64179,https://www.4s.com/en/ecatalog?part=64179&type...


In [6]:
crawler_status = 'ok'

crawler_status

'ok'

In [7]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [8]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_number_error.xlsx', index=False)

output_error

""
